<font size="7" face="monospace">Top 20</font>

<br>
<font size="5" face="monospace">
    資料來源：http://www.twse.com.tw/zh/page/trading/exchange/MI_INDEX20.html
</font>

<br>
<font size="5" face="monospace">
    不同公司比較其成交資訊
</font>

In [1]:
# %matplotlib inline
import requests
import pandas as pd
import multiprocessing as mp
from functools import partial
import matplotlib.pyplot as plt 

In [2]:
def url(y, m, d):
    
    y = str(y)
    m = str(m)
    d = str(d)
    
    #兩位數，若有空則補零
    m = m.zfill(2)
    d = d.zfill(2)
    
    url = 'http://www.twse.com.tw/exchangeReport/MI_INDEX20?response=csv&date=' + y + m + d
    table = pd.read_csv(url, encoding='big5')
    
    return table

In [3]:
def clean(table):
    
    #刪除不需要的欄位
    table_name = table.columns.values[0] #儲存標頭
    table = table.reset_index() #重設index和column names
    table.columns = ['排名', '證券代號', '證券名稱', '成交股數', '成交筆數' , '開盤價', '最高價', '最低價', '收盤價', '漲跌(+/-)', '漲跌價差', '最後揭示買價', '最後揭示賣價', 'temp']
    table.drop(['temp'], axis = 1, inplace = True) #將最後一行的NaN刪除
    table.drop([0, 21, 22, 23, 24], axis = 0, inplace = True) #將說明刪除
    table = table.reset_index(drop = True) #重設index   
    
    
#     #將string轉換為float
#     key = ['成交股數', '成交金額', '開盤價', '最高價' , '最低價', '收盤價', '漲跌價差', '成交筆數']
#     for i in range(len(key)):
#         table[key[i]] = table[key[i]].apply(lambda x: float(x.split()[0].replace(',', '')))
#         table[key[i]] = table[key[i]].astype(float)
    
#     #將string轉為西元，再轉為datetime
#     for i in range(len(table['日期'])):
#         day = table.loc[i, '日期']
#         day = day.replace('/', '-')
#         temp = str(int(day[0:3]) + 1911) #民國轉西元
#         day = day.replace(day[0:3], temp)
#         table.loc[i, '日期'] = day
#     table['日期'] = table['日期'].astype('datetime64[ns]')
    
    return (table_name, table)

In [4]:
def code(table):
    
    c = table['證券代號']
    
    for i in range(len(c)):
        c_temp = c[i] 
        
        #若證券代號為 ='xxxx'，將其改為正常格式
        if c_temp[0] == '=':
            c_temp = c_temp.replace(c_temp[0:2], '')
            c_temp = c_temp.replace(c_temp[6], '')
            c[i] = c_temp
            
    c = c.drop(c.index[0]) #刪除證券代號（文字），留下純代號
            
    return c

In [5]:
def title(name):
    
    name = name.replace(' ', '')
    temp = str(int(name[0:3]) + 1911) #民國轉西元
    name = name.replace(name[0:3], temp)
    
    return name

In [11]:
def main(year, month, day):

    table = url(year, month, day)
    table = clean(table)

    #標頭
    table_name = table[0]
    table_name = title(table_name)

    #表格
    table_data = table[1]

    #取得前20名證券代號
    top20 = code(table_data)

    table = [table_name, table_data, top20]